## Практическая работа № 2 "Разграничение доступа"

### Автор: Гордеев Александр Сергеевич КЭ-401

Вариант №2

Количество субъектов (пользователей): 4

Количество объектов доступа: 4

Импортируем необходимые классы и функции из модулей

In [149]:
from abc import ABC
from random import randint
from typing import Callable

Распишем класс со следующими правами в виде битовой маски:

- EMPTY - права отсутствуют
- READ - права для чтения
- WRITE - права на запись
- EXECUTE - права для запуска
- TRANSFER - права для передачи своих прав
- FULL - полные права доступа

In [150]:
class Permissions:
    EMPTY = 0b0000
    READ = 0b0001
    WRITE = 0b0010
    EXECUTE = 0b0100
    TRANSFER = 0b1000
    FULL = READ | WRITE | EXECUTE | TRANSFER

    @staticmethod
    def string(permissions: int) -> str:
        if permissions == Permissions.EMPTY:
            return "empty"
        if permissions == Permissions.FULL:
            return "full"

        rights = []
        if permissions & Permissions.READ:
            rights.append("read")
        if permissions & Permissions.WRITE:
            rights.append("write")
        if permissions & Permissions.EXECUTE:
            rights.append("execute")
        if permissions & Permissions.TRANSFER:
            rights.append("transfer")
        return ", ".join(rights)

Распишем классы системы:

- Object - базовый класс для объектов
- File - класс файлов, наследуемый от Object
- Device - класс устройств, наследуемый от Object
- User - класс пользователей

In [151]:
class Object(ABC):
    _counter = 0

    def __init__(self, name: str):
        self.name = name
        self.id = Object._counter
        Object._counter += 1

    def __eq__(self, other: object) -> bool:
        return isinstance(other, Object) and self.id == other.id

    def __hash__(self) -> int:
        return hash(self.id)

    def __str__(self) -> str:
        return self.name


class File(Object):
    def __init__(self, name: str):
        super().__init__(name)


class Device(Object):
    def __init__(self, name: str):
        super().__init__(name)


class User:
    def __init__(self, name: str):
        self.name = name
        self.is_admin = False

    def __eq__(self, other: object) -> bool:
        return isinstance(other, User) and self.name == other.name

    def __hash__(self) -> int:
        return hash(self.name)

    def __str__(self) -> str:
        return self.name

Реализуем класс для управления доступом к объектам системы

In [152]:
class AccessManager:
    def __init__(self):
        self._users: dict[str, User] = {}
        self._objects: dict[str, Object] = {}
        self._access_table: dict[tuple[User, Object], Permissions] = {}

    def add_user(self, user: User) -> None:
        self._users[user.name] = user
        for obj in self._objects.values():
            self._grant_permissions(user, obj)

    def add_object(self, obj: Object) -> None:
        self._objects[obj.name] = obj
        for user in self._users.values():
            self._grant_permissions(user, obj)

    def set_user_admin(self, user_name: str) -> None:
        user = self._users.get(user_name, None)
        user.is_admin = True
        for obj in self._objects.values():
            self._grant_permissions(user, obj)

    def get_user(self, user_name: str) -> User:
        return self._users.get(user_name, None)

    def get_object(self, obj_name: str) -> Object:
        return self._objects.get(obj_name, None)

    def check_permissions(self, user: User, obj: Object) -> Permissions:
        return self._access_table.get((user, obj), None)

    def check_permissions_by_str(self, user_name: str, obj_name: str) -> Permissions:
        return self.check_permissions(
            self.get_user(user_name), self.get_object(obj_name)
        )

    def get_user_permissions(self, user: User) -> list[tuple[Object, Permissions]]:
        return [
            (obj, self._access_table.get((user, obj), None))
            for obj in self._objects.values()
        ]

    def _grant_permissions(self, user: User, obj: Object) -> None:
        if user.is_admin:
            self._access_table[(user, obj)] = Permissions.FULL
        else:
            self._access_table[(user, obj)] = randint(0, 15)

Для демонстрации работы AccessManager реализуем класс для взаимодействия пользователя с системой

In [153]:
class Parser:
    def __init__(self):
        self._commands: dict[str, Callable] = {}

    def add_command(self, name: str, func: Callable) -> None:
        self._commands[name] = func

    def parse(self, input: str) -> None:
        if not input or len(input) == 0:
            return

        command, *args = input.split()
        if command in self._commands:
            self._commands[command](args)
        else:
            print(f"Unknown command: {command}")

    def get_commands(self) -> dict[str, Callable]:
        return self._commands


class SystemDemo:
    def __init__(self):
        self._is_exit = False
        self._manager = AccessManager()
        self._parser = Parser()
        self._init_manager()
        self._init_parser()

    def run(self) -> None:
        user: User = None
        while not user:
            inp = input("Enter username: ")
            if inp in ["quit", "exit"]:
                return
            user = self._authenticate(inp)
            if not user:
                print("Authentication failed! Try again...")

        self._greetings_message(user)

        while not self._is_exit:
            inp = input(f"{user}> ")
            if not inp or len(inp) == 0:
                continue
            self._parser.parse(f"{inp} {user.name}")

    def _init_manager(self):
        self._manager.add_user(User("Alice"))
        self._manager.add_user(User("Bob"))
        self._manager.add_user(User("Eve"))
        self._manager.add_user(User("root"))
        self._manager.set_user_admin("root")
        self._manager.add_object(File("file1"))
        self._manager.add_object(File("file2"))
        self._manager.add_object(Device("device1"))
        self._manager.add_object(Device("device2"))

    def _init_parser(self):
        self._parser.add_command("help", self._help)
        self._parser.add_command("quit", self._quit)
        self._parser.add_command("exit", self._quit)
        self._parser.add_command("read", self._read)
        # self._parser.add_command("write", self._write)
        # self._parser.add_command("execute", self._execute)
        # self._parser.add_command("transfer", self._transfer)

    def _authenticate(self, username: str) -> User:
        return self._manager.get_user(username)

    def _greetings_message(self, user: User) -> None:
        print("Authenticated user. Welcome to the system!")
        print("Your permissions:")
        perms = self._manager.get_user_permissions(user)
        for obj, rights in perms:
            print(f"{obj}: {Permissions.string(rights)}")

    def _help(self, _: list[str]) -> None:
        print("Commands:")
        for name in self._parser.get_commands().keys():
            print(f"  {name}")

    def _quit(self, _: list[str]) -> None:
        print("Bye!")
        self._is_exit = True

    def _read(self, args: list[str]) -> None:
        if len(args) < 2:
            print("Usage: read <object>")
            return

        obj = args[0]
        user = args[len(args) - 1]

        rights = self._manager.check_permissions_by_str(user, obj)
        if rights == None:
            print("User or object not found")
            return
        if rights & Permissions.READ == 0:
            print("Access denied")
            return

        print("Access granted")

Запускаем для демонстрации

In [154]:
demo = SystemDemo()
demo.run()

Authentication failed! Try again...
Authenticated user. Welcome to the system!
Your permissions:
file1: full
file2: full
device1: full
device2: full
User or object not found
Unknown command: rrrr
Usage: read <object>
User or object not found
User or object not found
Access granted
Bye!
